<a href="https://colab.research.google.com/github/TuragDev/Predicting-Crystal-System-with-ML-DL/blob/main/crystal_system_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install xgboost

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving magpie_testdataset.csv to magpie_testdataset.csv
Saving magpie_traindataset.csv to magpie_traindataset.csv
Saving Orbital_testdataset.csv to Orbital_testdataset.csv
Saving Orbital_traindataset.csv to Orbital_traindataset.csv
Saving Sine_testdataset.csv to Sine_testdataset.csv
Saving Sine_traindataset.csv to Sine_traindataset.csv
Saving Xrd_testdataset.csv to Xrd_testdataset.csv
Saving Xrd_traindataset.csv to Xrd_traindataset.csv


In [ ]:
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import sys
# In[3]:


from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV


Magpie Descriptor

In [ ]:
df= pd.read_csv('magpie_traindataset.csv')
df.shape

(584, 134)

In [ ]:
X_train=df.iloc[:,1:132]
y_train=df['crystal_system']

y_train

0         triclinic
1         triclinic
2      orthorhombic
3         triclinic
4         triclinic
           ...     
579       triclinic
580      monoclinic
581       triclinic
582       triclinic
583      monoclinic
Name: crystal_system, Length: 584, dtype: object

In [ ]:
df=pd.read_csv('magpie_testdataset.csv')
X_test=df.iloc[:,1:132]

y_test=df['crystal_system']

y_test

0         triclinic
1         triclinic
2      orthorhombic
3         triclinic
4         triclinic
           ...     
141      monoclinic
142       triclinic
143       triclinic
144      monoclinic
145       triclinic
Name: crystal_system, Length: 146, dtype: object

In [ ]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

In [ ]:
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy


In [ ]:
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

100%|██████████| 50/50 [03:04<00:00,  3.70s/trial, best loss: -0.684931506849315]


In [ ]:
print("Best Hyperparameters:", best)

Best Hyperparameters: {'colsample_bytree': 0.78410034442625, 'gamma': 3.120794514948667, 'learning_rate': 1.478490730409349, 'max_depth': 5.0, 'min_child_weight': 4.0, 'n_estimators': 290.0, 'reg_alpha': 2.083835144238994, 'reg_lambda': 2.7486578374783077, 'subsample': 0.6911229636778327}


In [ ]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 290,
    'learning_rate': 1.478490730409349,
    'gamma': 3.120794514948667,
    'max_depth': 5,
    'min_child_weight': 4,
    'colsample_bytree':  0.78410034442625,
    'subsample': 0.6911229636778327,
    'reg_alpha': 2.083835144238994,
    'reg_lambda': 2.7486578374783077
}

In [ ]:
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Magpie_Bayes_crystal = label_encoder.inverse_transform(pred)
Magpie_Bayes_crystal

Mean Accuracy: 0.6185322723253758
Accuracy on Training data:  0.6421232876712328
Accuracy on Test data:  0.678082191780822


array(['triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'monoclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'cubic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'orthorhombic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'monocli

Sine descriptor

In [ ]:
df= pd.read_csv('Sine_traindataset.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [ ]:
df=pd.read_csv('Sine_testdataset.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [ ]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [01:21<00:00,  1.62s/trial, best loss: -0.28187919463087246]
Best Hyperparameters: {'colsample_bytree': 0.9450208688534998, 'gamma': 4.224227358373126, 'learning_rate': 13.517959034917455, 'max_depth': 6.0, 'min_child_weight': 7.0, 'n_estimators': 12.0, 'reg_alpha': 3.1602816817120565, 'reg_lambda': 2.171173585105933, 'subsample': 0.6246441675497391}


In [ ]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 17,
    'learning_rate': 13.517959034917455,
    'gamma':  4.224227358373126,
    'max_depth': 6,
    'min_child_weight': 7,
    'colsample_bytree': 0.9450208688534998,
    'subsample': 0.6246441675497391,
    'reg_alpha': 3.1602816817120565,
    'reg_lambda': 2.171173585105933
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.4855784313725491


In [ ]:
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Sine_Bayes_crystal = label_encoder.inverse_transform(pred)
Sine_Bayes_crystal

Accuracy on Training data:  0.6845637583892618
Accuracy on Test data:  0.1476510067114094


array(['trigonal', 'trigonal', 'trigonal', 'trigonal', 'monoclinic',
       'trigonal', 'monoclinic', 'trigonal', 'trigonal', 'trigonal',
       'trigonal', 'trigonal', 'trigonal', 'trigonal', 'trigonal',
       'monoclinic', 'trigonal', 'trigonal', 'trigonal', 'monoclinic',
       'trigonal', 'trigonal', 'trigonal', 'monoclinic', 'trigonal',
       'monoclinic', 'trigonal', 'trigonal', 'trigonal', 'trigonal',
       'monoclinic', 'trigonal', 'trigonal', 'monoclinic', 'trigonal',
       'trigonal', 'monoclinic', 'trigonal', 'trigonal', 'monoclinic',
       'trigonal', 'trigonal', 'monoclinic', 'monoclinic', 'trigonal',
       'trigonal', 'trigonal', 'trigonal', 'monoclinic', 'trigonal',
       'monoclinic', 'trigonal', 'monoclinic', 'monoclinic', 'trigonal',
       'trigonal', 'monoclinic', 'trigonal', 'trigonal', 'monoclinic',
       'monoclinic', 'trigonal', 'trigonal', 'trigonal', 'trigonal',
       'monoclinic', 'monoclinic', 'trigonal', 'trigonal', 'trigonal',
       'monoclinic',

OFM Descriptor

In [6]:
df= pd.read_csv('Orbital_traindataset.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [8]:
df=pd.read_csv('Orbital_testdataset.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [9]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)

In [10]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [11]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

In [12]:
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [10:27<00:00, 12.55s/trial, best loss: -0.6711409395973155]
Best Hyperparameters: {'colsample_bytree': 0.9692421016481573, 'gamma': 1.5242388920532575, 'learning_rate': 1.4157421779156725, 'max_depth': 3.0, 'min_child_weight': 4.0, 'n_estimators': 356.0, 'reg_alpha': 0.03379904545550308, 'reg_lambda': 3.119057643751944, 'subsample': 0.6578039493835993}


In [13]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 356,
    'learning_rate': 1.4157421779156725,
    'gamma':  1.5242388920532575,
    'max_depth': 3,
    'min_child_weight': 4,
    'colsample_bytree':  0.9692421016481573,
    'subsample':0.6578039493835993,
    'reg_alpha': 0.03379904545550308,
    'reg_lambda':3.119057643751944
}



In [14]:
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Orbital_Bayes_crystal = label_encoder.inverse_transform(pred)
Orbital_Bayes_crystal

Mean Accuracy: 0.6226890756302522
Accuracy on Training data:  0.8941176470588236
Accuracy on Test data:  0.6308724832214765


array(['triclinic', 'monoclinic', 'cubic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'cubic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'monoclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'cubic', 'triclinic',
       'triclinic', 'monoclinic', 'tetragonal', 'triclinic', 'triclinic',
       'monoclinic', 'orthorhombic', 'triclinic', 'monoclinic',
       'monoclinic', 'cubic', 'triclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',


Xrd

In [15]:
df= pd.read_csv('Xrd_traindataset.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [18]:
df=pd.read_csv('Xrd_testdataset.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [19]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [20]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

In [21]:
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [03:07<00:00,  3.76s/trial, best loss: -0.6845637583892618]
Best Hyperparameters: {'colsample_bytree': 0.7520939618416126, 'gamma': 0.8677428116703306, 'learning_rate': 1.2504114238629624, 'max_depth': 4.0, 'min_child_weight': 4.0, 'n_estimators': 127.0, 'reg_alpha': 0.6080585062080629, 'reg_lambda': 3.6095428943920727, 'subsample': 0.9711079568490482}


In [22]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 127,
    'learning_rate': 1.2504114238629624,
    'gamma':  0.8677428116703306,
    'max_depth': 4,
    'min_child_weight': 4,
    'colsample_bytree':  0.7520939618416126,
    'subsample': 0.9711079568490482,
    'reg_alpha': 0.6080585062080629,
    'reg_lambda': 3.6095428943920727
}


In [23]:
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Xrd_Bayes_crystal = label_encoder.inverse_transform(pred)
Xrd_Bayes_crystal

Mean Accuracy: 0.7001512605042016
Accuracy on Training data:  0.988255033557047
Accuracy on Test data:  0.6711409395973155


array(['monoclinic', 'triclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'orthorhombic', 'triclinic', 'orthorhombic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'orthorhombic', 'triclinic', 'triclinic',
       'monoclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'monoclinic', 'hexagonal', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclin